```
p_паттерн парсит текст (str, pos), 
    вызывая другие паттерны, возвращающие древовидные структуры
    эти древовидные структуры передает одному из правил
        (остальные вызовы правил закомментированы 
            - в рамках одного текста каждый паттерн имеет ровно 1 смысл)
    и возвращает (pos, результат этого правила), помещенный в массив
    
    /*пока на ошибках парсинга концентрировать не будем*/
    если ничего не удалось распарсить, ???возвращаемый массив будет пустым???
    если удалось распарсить несколько вариантов - в массиве будет насколько вариантов
        сначала парсятся все исключения
        потом парсятся все обычные варианты (если нет исключений)
```

```
r_правило получает список древовидных структур
    обрабатывает их по определенному правилу
    возвращает древовидную структуру
```

```
древовидная структура - map, со следующими ключами
    type: 'noun'/'adj'/'verb'/'numeric'/...
    постоянные параметры (для сущ.: род, число)
    переменные параметры (для сущ.: падеж)
    talk: массив древовидных структур
        который также может содержать 1 слово (для сущ. в и.п.)
```

```
show древовидная структура
    превращает древовидную структуру в строку
```

```
для каждого типа элемента древовидной структуры будет map
    где каждой строке (для сущ. - слово в и.п.) будет соответствовать
        структура данных, которая вместе в переменными параметрами передается в 
        sh_функция для этого типа элемента древовидной структуры
            которая будет возвращать слово в соотв. форме (для сущ. в соотв. падеже)
```

# Общее

In [3]:
def throw(ex):
    raise ex

In [10]:
def en2ru(s): # main
    rezs=[]
    
    rezs+=[res for pos,res in p_noun(s,0) if pos==len(s)]
    # добавить дочитывание точки и остаточных пробелов
        
    return [str(i) for i in rezs]

## Паттерны парсинга

In [71]:
def p_spcs(str,pos):
    pos1=pos
    while pos1<len(str) and (str[pos1]==' ' or str[pos1]=='\n' or str[pos1]=='\t') :
        pos1+=1
    return [] if pos1==pos else [(pos1,'')]

In [130]:
def p_word(str,pos):
    pos1=pos
    while pos1<len(str) and (str[pos1]>='a' and str[pos1]<='z' or \
                             str[pos1]>='A' and str[pos1]<='Z') :
        pos1+=1
    return [] if pos1==pos else [(pos1,str[pos:pos1])]

In [79]:
def p_seq(str,pos,*patterns):
    rezs=[]
    for patt in patterns:
        tmp=patt(str,pos)
        if len(tmp)>1 : raise NotImplementedError()
        if len(tmp)==0 : return []
        (pos,tmp)=tmp[0]
        rezs.append(tmp)
    return [(pos,rezs)]

# Словарь

## Классы отображения

In [93]:
class Struct:
    type=None
    talk=[]
    def __str__(self):
        ' '.join(str(i) for i in self.talk)
        # убрать пробелы перед знаками препинания и сделать заглавные буквы

In [155]:
class StNoun(Struct):
    def __init__(self,odush,rod,chis,pad,talk):
        self.type='noun'
        
        if type(odush)!=bool : raise TypeError('odush must be bool')
        self.odush=odush
        
        if rod!='m' and rod!='g' and rod!='s' : raise TypeError('rod must be m or g or s')
        self.rod=rod
        
        if chis!='ed' and chis!='mn' : raise TypeError('chis must be ed or mn')
        self.chis=chis
        
        if pad!='ip' and pad!='rp' and pad!='dp' and pad!='vp' and pad!='tp' and pad!='pp' : 
            raise TypeError('pad must be ip, rp, dp, vp, tp or pp')
        self.pad=pad
        
        self.talk=talk
        
    def __repr__(self):
        return '{type:'+self.type+',odush:'+str(self.odush)+',rod:'+self.rod+\
            ',chis:'+self.chis+',pad:'+self.pad+',talk:'+\
            repr(self.talk)+'}'
    
    def __str__(self):
        return ' '.join(str(i) if isinstance(i,Struct) else show_noun_map[i](self) for i in self.talk)

def show_noun1(pad,word,ip,rp,dp,vp,tp,pp):
    if pad=='ip' : return word+ip
    if pad=='rp' : return word+rp
    if pad=='dp' : return word+dp
    if pad=='vp' : return word+vp
    if pad=='tp' : return word+tp
    if pad=='pp' : return word+pp
    raise InternalError()


In [153]:
class StAdj(Struct):
    def __init__(self,odush,rod,chis,pad,talk):
        self.type='adj'
        
        if type(odush)!=bool : raise TypeError('odush must be bool')
        self.odush=odush
        
        if rod!='m' and rod!='g' and rod!='s' : raise TypeError('rod must be m or g or s')
        self.rod=rod
        
        if chis!='ed' and chis!='mn' : raise TypeError('chis must be ed or mn')
        self.chis=chis
        
        if pad!='ip' and pad!='rp' and pad!='dp' and pad!='vp' and pad!='tp' and pad!='pp' : 
            raise TypeError('pad must be ip, rp, dp, vp, tp or pp')
        self.pad=pad
        
        self.talk=talk
        
    def __repr__(self):
        return '{type:'+self.type+',odush:'+str(self.odush)+',rod:'+self.rod+\
            ',chis:'+self.chis+',pad:'+self.pad+',talk:'+\
            repr(self.talk)+'}'
    
    def __str__(self):
        return ' '.join(str(i) if isinstance(i,Struct) else show_adj_map[i](self) for i in self.talk)

def show_adj1(st,word,ends):
    if st.chis=='mn' :
        (ip,rp,dp,vp,tp,pp,sh)=ends['mn']
    else:
        (ip,rp,dp,vp,tp,pp,sh)=ends[st.rod]
        
    if   st.pad=='ip' : rez=ip
    elif st.pad=='rp' : rez=rp
    elif st.pad=='dp' : rez=dp
    elif st.pad=='vp' : rez=vp[0] if st.odush else vp[1]
    elif st.pad=='tp' : rez=tp
    elif st.pad=='pp' : rez=pp
    elif st.pad=='sh' : rez=sh
    else: raise InternalError()
        
    return word+rez


## Отображение

In [154]:
show_adj_map={
    'летучий'  :lambda st: show_adj1(st,'летуч' ,
        {
            'm' :('ий','его','ему',('его','ий'),'им' ,'ем','' ),
            's' :('ее','его','ему',('ий', 'ий'),'им' ,'ем','е'),
            'g' :('ая','ей' ,'ей' ,('ую', 'ую'),'ей' ,'ей','а'),
            'mn':('ие','их' ,'им' ,('их', 'ие'),'ими','их','и'),
        }
    ) ,
}

In [156]:
show_noun_map={
    'кот'  :lambda st: show_noun1(st.pad,'кот' ,'' ,'а','у','а','ом','е') ,
    'урок' :lambda st: show_noun1(st.pad,'урок','' ,'а','у','' ,'ом','е') ,
    'кошка':lambda st: show_noun1(st.pad,'кошк','а','и','е','у','ой','е') ,
    'крыса':lambda st: show_noun1(st.pad,'крыс','а','ы','е','у','ой','е') ,
    'мышь' :lambda st: show_noun1(st.pad,'мыш' ,'ь','и','и','ь','ью','и') ,
}

## Правила

In [150]:
def r_kot():    return StNoun(True,'m','ed','ip',['кот'])
def r_koshka(): return StNoun(True,'g','ed','ip',['кошка'])
def r_krysa():  return StNoun(True,'g','ed','ip',['крыса'])
def r_mysh():   return StNoun(True,'g','ed','ip',['мышь'])
def r_urok():   return StNoun(False,'m','ed','ip',['урок'])

In [125]:
def r_letuchij(): return StAdj(True,'g','ed','ip',['летучий'])

In [104]:
def r_a_noun(a,n): return n
def r_adj_noun(a,n): 
    a.rod=n.rod
    a.odush=n.odush
    n.talk.insert(0,a)
    return n

## Словарь

In [49]:
def p_ad_an_article(str,pos):

    if str[pos:pos+2]=='an' :
        return [(pos+2,'')]
        #return [(pos+2),r_nekotoryj()]
    if str[pos:pos+1]=='a' :
        return [(pos+1,'')]
        #return [(pos+1),r_nekotoryj()]
        
    return []

In [158]:
dict_noun={
    'cat':  r_koshka(),
    #'cat':  r_kot(),
    
    #'rat':  r_mysh(),
    'rat':  r_krysa(),
    
    'bat':  r_adj_noun(r_letuchij(),r_mysh()),
    
    'lesson': r_urok(),
}

## Паттерны

In [45]:
def p_adj(str,pos):
    rezs=[]
    
    rezs+=p_ad_an_article(str,pos)
        
    return rezs

In [135]:
def p_noun(str,pos):
    rezs=[]
    
    #исключение словосочетания
    if 1:
        rezs+=[\
            (pos1,r_a_noun(a,n)) \
            #(pos1,r_nekotoryj_noun(a,s)) \
                for (pos1,[a,s,n]) in p_seq(str,pos,p_ad_an_arcticle,p_spcs,p_noun) ]
    
    if len(rezs)>0 : return rezs
    #словосочетание
    if 1:
        rezs+=[\
            (pos1,r_adj_noun(a,n)) \
            #(pos1,r_nekotoryj_noun(a,s)) \
                for (pos1,[a,s,n]) in p_seq(str,pos,p_adj,p_spcs,p_noun) ]
    
    if len(rezs)>0 : return rezs
    #слово

    tmp=[]
    for tpos,tword in p_word(str,pos):
        if tword in dict_noun:
            tmp.append((tpos,dict_noun[tword]))
    rezs+=tmp
        
    return rezs

# Тесты

```
контроль версий
словарь прилагательных
p_adj_noun
p_noun_and_noun
p_simple_noun
StWNoun
getter & setter падежей сущ
числительные: род число падеж одуш - пока будем использовать как прилагательные, 
    но число числит. и число сущ. должны совпадать - проверка во время парсинга, 
        иначе исключение
```

In [136]:
p_noun('cat',0)

[(3, {type:noun,odush:True,rod:g,chis:ed,pad:ip,talk:['кошка']})]

In [138]:
p_spcs(' ',0)

[(1, '')]

In [100]:
[]

True

## Lesson 1

In [159]:
en2ru('cat')

['кошка']

In [140]:
en2ru('a cat')

['кошка']

In [149]:
en2ru('rat')

['крыса']

In [145]:
en2ru('a bat')

['летучая мышь']

In [160]:
en2ru('lesson')

['урок']

## Lesson 2